In [228]:
! pip install librosa

In [229]:
import os
import librosa
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [230]:
# Read csv file 
# Data was downloaded from https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
#There are audio files and csv file. For this part csv with 30 sec music features was used.
path = "mldata_v2.csv"
df = pd.read_csv(path)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav_1,66150,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829707,1972.334258,117272.640573,...,39.547077,-3.230046,36.606857,0.696385,37.766136,-5.035945,33.668550,-0.239585,43.818886,blues
1,blues.00000.wav_2,66150,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297514,2010.751494,65940.666037,...,64.819780,-6.025473,40.548813,0.127131,51.048943,-2.808956,97.221504,5.771881,60.360348,blues
2,blues.00000.wav_3,66150,0.347746,0.092495,0.130895,0.004552,1790.722358,110071.206762,2088.184750,73391.498088,...,68.306790,-1.714475,28.136944,2.329553,47.211426,-1.925621,52.922430,2.466996,33.163998,blues
3,blues.00000.wav_4,66150,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936309,1967.920582,79805.901501,...,48.543200,-3.786986,28.419546,1.153315,35.682700,-3.501979,50.610344,3.580636,32.325880,blues
4,blues.00000.wav_5,66150,0.335481,0.088482,0.142370,0.001734,1634.465076,77425.419156,1954.633566,57359.695597,...,30.829544,0.635798,44.645557,1.591107,51.415867,-3.364908,26.421090,0.501504,29.109533,blues


In [231]:
df.columns

Index(['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var',
       'spectral_flatness_mean', 'spectral_flatness_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var',
       'tonnetz_mean', 'tonnetz_var', 'chroma_cens_mean', 'chroma_cens_var',
       'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var',
       'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean',
       'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var',
       'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean',
       'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var',
       'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean',
       'mfcc15_var', 'mfcc16_mean', 'mfc

In [232]:
df.shape[1]

66

In [233]:
# Prepare features (X) and target (y)
df.drop(["filename"], axis=1, inplace=True)  # Drop unnecessary columns
X = df.drop("label", axis=1)  # Features
y = df["label"]  # Target label

In [234]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [235]:
# One-Hot Encoding
y_categorical = to_categorical(y_encoded)
y_categorical

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [236]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [237]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y_categorical,random_state=1)

#1 Neural Network Model

In [238]:
# Create a Sequential model
model = Sequential()

In [239]:
# Input layer and first hidden layer
model.add(Dense(units=128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dropout(0.3))

c:\Users\weiwei\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [240]:
# Second hidden layer
model.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))

In [241]:
# Third hidden layer
model.add(Dense(units=32,activation='relu'))
model.add(Dropout(0.3))

In [242]:
# Output layer(number of units matches the number of the classes)
model.add(Dense(units=y_categorical.shape[1], activation='softmax'))

In [243]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [244]:
# train the NN model

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
NN_model=model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stop])


Epoch 1/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2041 - loss: 2.9571 - val_accuracy: 0.5377 - val_loss: 1.9525
Epoch 2/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4330 - loss: 2.0136 - val_accuracy: 0.6333 - val_loss: 1.4632
Epoch 3/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5069 - loss: 1.6546 - val_accuracy: 0.6673 - val_loss: 1.2360
Epoch 4/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5654 - loss: 1.4517 - val_accuracy: 0.6987 - val_loss: 1.0928
Epoch 5/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6202 - loss: 1.2743 - val_accuracy: 0.7034 - val_loss: 1.0221
Epoch 6/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6536 - loss: 1.1615 - val_accuracy: 0.7255 - val_loss: 0.9850
Epoch 7/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6598 - loss: 1.1303 - val_accuracy: 0.7348 - val_loss: 0.9120
Epoch 8/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6701 - loss: 1.0754 - val_accu

In [245]:
# Evaluate the NN model
test_loss, test_accuracy = model.evaluate(X_test, y_test,verbose=1)
print(test_accuracy)

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8879 - loss: 0.4761
0.8806089758872986


In [246]:
# Predict class probabilities for the test set
y_pred = model.predict(X_test)

# Convert predictions from probabilities to class indices
y_pred_classes = y_pred.argmax(axis=1)

# Convert one-hot encoded y_test back to class indices
y_test_classes = y_test.argmax(axis=1)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [247]:
# Classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))




Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       258
           1       0.93      0.96      0.95       242
           2       0.90      0.84      0.87       263
           3       0.88      0.79      0.83       234
           4       0.84      0.87      0.86       242
           5       0.85      0.92      0.88       248
           6       0.93      0.97      0.95       264
           7       0.91      0.86      0.88       257
           8       0.84      0.85      0.85       245
           9       0.84      0.80      0.82       243

    accuracy                           0.88      2496
   macro avg       0.88      0.88      0.88      2496
weighted avg       0.88      0.88      0.88      2496



#2  XGBoost Model

In [248]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, random_state=1)

In [249]:
# Create the XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100)

In [250]:
# Train the XGBoost model
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [251]:
# Make predictions
y_pred = xgb_model.predict(X_test)


In [252]:
# Evaluate the model
test_accuracy = np.mean(y_pred == y_test)
test_accuracy


0.9094551282051282

In [253]:
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       258
           1       0.95      0.97      0.96       242
           2       0.87      0.88      0.87       263
           3       0.91      0.88      0.90       234
           4       0.93      0.90      0.92       242
           5       0.91      0.92      0.92       248
           6       0.95      0.95      0.95       264
           7       0.92      0.89      0.91       257
           8       0.91      0.89      0.90       245
           9       0.84      0.86      0.85       243

    accuracy                           0.91      2496
   macro avg       0.91      0.91      0.91      2496
weighted avg       0.91      0.91      0.91      2496

